## Purpose - To scrape data on NBER Summer Institute Participants, their affiliation and their authorship details within the world of NBER WPs

### Import Packages

In [1]:
import pandas as pd 
import os 
import numpy as np 
import re
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import math
from selenium.common.exceptions import NoSuchElementException

In [2]:
EXE_PATH = r"C:\Users\nik596\Downloads\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(executable_path=EXE_PATH)

C:\Users\nik596\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [3]:
base = "https://www.nber.org"

In [4]:
html1 = urlopen(base + "/conferences/summer-institute")
soup1 = BeautifulSoup(html1, 'lxml')

In [13]:
dic = {}
count = 0
links = soup1.findAll('a')
for link in links:
    if 'Summer Institute' in link.text and ':' not in link.text and '2022' not in link.text:
        conf_link = link.get('href')
        conf_text = link.text.strip()
        print(conf_text)
        html2 = urlopen(base + conf_link)
        soup2 = BeautifulSoup(html2, 'lxml')
        wkshps = soup2.findAll('a')
        for wkshp in wkshps:
            if 'SI 20' in wkshp.text and '2022' not in wkshp.text:
                wkshp_text = wkshp.text.strip()
                wkshp_link = wkshp.get('href')
                print(wkshp_text)
                html3 = urlopen(base + wkshp_link)
                soup3 = BeautifulSoup(html3, 'lxml')
                participants = soup3.find('div', class_ = 'promo-grid__promos').find_all('div', class_ = 'name-promo')
                for participant in participants:
                    name = participant.find('span', class_ = 'name-promo__name').text.strip().replace(",", "")
                    affil = participant.find('span', class_ = 'name-promo__title').text.strip() if participant.find('span', class_ = 'name-promo__title') is not None else ""
                    paper_in_nber = participant.find('a') is not None 
                    dic[count] = {"Conference": conf_text, 
                                    "Workshop": wkshp_text,
                                    "Name": name,
                                     "Affiliation": affil,
                                      "Paper In NBER": paper_in_nber,
                                     "member": "NBER" in affil}
                    if paper_in_nber:
                        participant_link = participant.find('a').get('href')
                        dic[count]['participant link'] = participant_link
                    count+=1

Summer Institute 2021
SI 2021 Inequality and the Macroeconomy
SI 2021 Corporate Finance
SI 2021 Capital Markets and the Economy
SI 2021 International Trade & Investment
SI 2021 Development of the American Economy
SI 2021 Monetary Economics
SI 2021 Impulse and Propagation Mechanisms
SI 2021 International Trade & Macroeconomics
SI 2021 International Asset Pricing
SI 2021 Forecasting & Empirical Methods
SI 2021 Risks of Financial Institutions
SI 2021 Macro, Money and Financial Markets
SI 2021 International Finance & Macroeconomics
SI 2021 International Finance and Macroeconomic Data Sources
SI 2021 Asset Pricing
SI 2021 Economic Growth
SI 2021 Dynamic Equilibrium Models
SI 2021 Behavioral Macro
SI 2021 Entrepreneurship
SI 2021 Macroeconomics Within and Across Borders
SI 2021 Conference on Research in Income and Wealth
SI 2021 Political Economy
SI 2021 Micro Data and Macro Models
SI 2021 Macroeconomics and Productivity
SI 2021 Innovation
SI 2021 Public Economics
SI 2021 Productivity, Devel

In [14]:
df=pd.DataFrame.from_dict(dic).T
df.to_excel(r'C:\Users\nik596\Documents\GitHub\NBER\SI_participants.xlsx',engine='xlsxwriter')

In [15]:
df

,Conference,Workshop,Name,Affiliation,Paper In NBER,member,participant link
0,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Harun Alp,Federal Reserve Board,True,False,/people/harun_alp
1,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Charlotte Bartels,DIW Berlin,True,False,/people/charlotte_bartels
2,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Roland Bénabou,Princeton University and NBER,True,True,/people/roland_benabou
3,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Corina Boar,New York University and NBER,True,True,/people/corina_boar
4,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Timo Boppart,"IIES, Stockholm University",True,False,/people/timo_boppart
...,...,...,...,...,...,...,...
60564,Summer Institute 2012,SI 2012 Economics of Crime Working Group,Ebonya L. Washington,Yale University and NBER,True,True,/people/ebonya_washington
60565,Summer Institute 2012,SI 2012 Economics of Crime Working Group,Andrea Weber,Central European University,True,False,/people/andrea_weber
60566,Summer Institute 2012,SI 2012 Economics of Crime Working Group,Christopher Winship,Harvard University,True,False,/people/christopher_winship
60567,Summer Institute 2012,SI 2012 Economics of Crime Working Group,Justin Wolfers,University of Michigan and NBER,True,True,/people/justin_wolfers


In [ ]:
df = pd.read_excel(r'C:\Users\nik596\Documents\GitHub\NBER\SI_participants_collab.xlsx')
last_conf = list(df['Conference'])[-1]
last_wkshp = list(df['Workshop'])[-1]
last_name = list(df['Name'])[-1]
dic = {}
count = 0
errors = []
last_conf_found = 0
last_wkshp_found = 0
last_name_found = 0
links = soup1.findAll('a')
for link in links:
    if 'Summer Institute' in link.text and ':' not in link.text and '2022' not in link.text:
        conf_link = link.get('href')
        conf_text = link.text.strip()
        if conf_text != last_conf and last_conf_found == 0:
            continue
        if conf_text == last_conf:
            last_conf_found = 1
        print(conf_text)
        html2 = urlopen(base + conf_link)
        soup2 = BeautifulSoup(html2, 'lxml')
        wkshps = soup2.findAll('a')
        for wkshp in wkshps:
            if 'SI 20' in wkshp.text and '2022' not in wkshp.text:
                wkshp_text = wkshp.text.strip()
                wkshp_link = wkshp.get('href')
                if wkshp_text != last_wkshp and last_wkshp_found == 0:
                    continue
                if wkshp_text == last_wkshp:
                    last_wkshp_found = 1
                print(wkshp_text)
                html3 = urlopen(base + wkshp_link)
                soup3 = BeautifulSoup(html3, 'lxml')
                participants = soup3.find('div', class_ = 'promo-grid__promos').find_all('div', class_ = 'name-promo')
                for participant in participants:
                    name = participant.find('span', class_ = 'name-promo__name').text.strip().replace(",","")
                    if name != last_name and last_name_found == 0:
                        continue
                    if name == last_name:
                        last_name_found = 1
                    print(name)
                    affil = participant.find('span', class_ = 'name-promo__title').text.strip() if participant.find('span', class_ = 'name-promo__title') is not None else ""
                    paper_in_nber = participant.find('a') is not None 
                    if not paper_in_nber:
                        dic[count] = {"Conference": conf_text, 
                                                        "Workshop": wkshp_text,
                                                        "Name": name,
                                                         "Affiliation": affil,
                                                         "Paper in NBER": paper_in_nber,
                                                          "member": "NBER" in affil}
                        count+=1
                        continue
                    else:
                        participant_link = participant.find('a').get('href')
                        browser.get(base + participant_link)
                        browser.implicitly_wait(30)
                        html4 = browser.page_source
                        soup4 = BeautifulSoup(html4, 'lxml')
                        temp = soup4
                        while temp == soup4:
                            temp = soup4
                            html4 = browser.page_source
                            soup4 = BeautifulSoup(html4, 'lxml')
                        text = browser.find_element_by_class_name("search__info-results").text
                        if "OF" in text:
                            pages = int(soup4.find_all('button')[-2].text)
                        elif "NO" in text:
                            dic[count] = {"Conference": conf_text, 
                                                        "Workshop": wkshp_text,
                                                        "Name": name,
                                                          "participant link": participant_link,
                                                         "Affiliation": affil,
                                                          "Paper in NBER": paper_in_nber,
                                                         "member": "NBER" in affil}
}
                            count+=1
                            continue
                        else:
                            pages = int(soup4.find_all('button')[-1].text)
                        participant_link = participant_link.replace("page=1", "page={}")
                        for i in range(1, pages + 1):
                            participant_link = participant_link.format(str(i))
                            
                            browser.get(base + participant_link)
                            browser.implicitly_wait(30)
    
                            paper_links = []
                            for element in browser.find_elements_by_xpath('//a[contains(@href,"/papers/")]'):
                                paper_link = element.get_attribute('href')
                                paper_links.append(paper_link)
                            paper_links = set(paper_links)
                            
                            for paper_link in paper_links:
                                try:
                                    html = urlopen(paper_link)
                                    soup = BeautifulSoup(html, 'lxml')
                                    paper_name = soup.find('h1', class_ = 'page-header__title').text.strip()
                                    paper_month = soup.find('time').text
                                    
                                    temp = soup.find('div', class_ = 'page-header__citation-item').text.strip()
                
                                    
                                    authors = soup.find_all('div', class_ = 'page-header__authors')[0].find_all('a')
                                    
                                    num = 1
                                    dic[count] = {"Conference": conf_text, 
                                                        "Workshop": wkshp_text,
                                                        "Name": name,
                                                          "participant link": participant_link,
                                                         "Affiliation": affil,
                                                         "Paper in NBER": paper_in_nber,
                                                          "member": "NBER" in affil,
                                                         "paper": paper_name,
                                                         "paper month": paper_month,
                                                         "paper number": temp[temp.find('Paper') + 6:]}
                                    """for author in authors:
            
                                        author_name = author.text
                                        
                                        dic[count]['author ' + str(num)] = author_name
            
                                        author_link = "https://www.nber.org" + author.get('href')
                                        html = urlopen(author_link)
                                        soup = BeautifulSoup(html, 'lxml')
            
                                        author_position = soup.find_all('div', class_ = 'person-header__title')[0].text.strip()
                                        dic[count]['author position ' + str(num)] = author_position
            
                                        author_affiliation = soup.find_all('div', class_ = 'person-header__summary')[0].text.strip()
                                        dic[count]['author affiliation ' + str(num)] = author_affiliation
                    
                                        author_address = soup.find_all('div', 'contact-card__social')[-1].text.strip().replace('\n', ' ')
                                        dic[count]['author address ' + str(num)] = author_address
                                        dic[count]['author num'] = num
                                        num+=1"""
                                    count+=1
                                except:
                                    errors.append(paper_link) 

In [15]:
df2=pd.DataFrame.from_dict(dic).T
df = df.append(df2, ignore_index=True)
df = df.rename(columns = {"member": "Paper in NBER"})
df["member"] = df["Affiliation"].apply(lambda x: "NBER" in x)
df

,Unnamed: 0,Conference,Workshop,Name,Affiliation,member,paper,paper month,paper number
0,0.0,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Harun Alp,Federal Reserve Board,True,"Innovation, Reallocation and Growth",April 2013,18993
1,1.0,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Harun Alp,Federal Reserve Board,True,Lack of Selection and Limits to Delegation: Fi...,January 2016,21905
2,2.0,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Charlotte Bartels,DIW Berlin,True,Long-Term Effects of Equal Sharing: Evidence f...,December 2020,28230
3,3.0,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Roland Bénabou,Princeton University and NBER,True,Laws and Norms,November 2011,17579
4,4.0,Summer Institute 2021,SI 2021 Inequality and the Macroeconomy,Roland Bénabou,Princeton University and NBER,True,Belief in a Just World and Redistributive Poli...,March 2005,11208
...,...,...,...,...,...,...,...,...,...
5539,NaN,Summer Institute 2021,SI 2021 Capital Markets and the Economy,Paul S. Willen,Federal Reserve Bank of Boston and NBER,True,Collateralized Borrowing and Life-Cycle Portfo...,June 2006,12309
5540,NaN,Summer Institute 2021,SI 2021 Capital Markets and the Economy,Paul S. Willen,Federal Reserve Bank of Boston and NBER,True,Do Lenders Still Discriminate? A Robust Approa...,August 2021,29142
5541,NaN,Summer Institute 2021,SI 2021 Capital Markets and the Economy,Paul S. Willen,Federal Reserve Bank of Boston and NBER,True,"Can't Pay or Won't Pay? Unemployment, Negative...",October 2015,21630
5542,NaN,Summer Institute 2021,SI 2021 Capital Markets and the Economy,Paul S. Willen,Federal Reserve Bank of Boston and NBER,True,The Time-Varying Price of Financial Intermedia...,August 2017,23706


In [16]:
df.to_excel(r'C:\Users\nik596\Documents\GitHub\NBER\SI_participants_collab.xlsx',engine='xlsxwriter')